# Code Search on Kubeflow

This notebook implements an end-to-end Semantic Code Search on top of [Kubeflow](https://www.kubeflow.org/) - given an input query string, get a list of code snippets semantically similar to the query string.

**NOTE**: If you haven't already, see [kubeflow/examples/code_search](https://github.com/kubeflow/examples/tree/master/code_search) for instructions on how to get this notebook,.

## Install dependencies

Let us install all the Python dependencies. Note that everything must be done with `Python 2`. This will take a while the first time.

### Verify Version Information

In [ ]:
%%bash

echo "Pip Version Info: " && python2 --version && python2 -m pip --version && echo
echo "Google Cloud SDK Info: " && gcloud --version && echo
echo "Ksonnet Version Info: " && ks version && echo
echo "Kubectl Version Info: " && kubectl version

### Install Pip Packages

In [ ]:
! python2 -m pip install -U pip

In [ ]:
# Code Search dependencies
! python2 -m pip install --user https://github.com/kubeflow/batch-predict/tarball/master
! python2 -m pip install --user -r src/requirements.txt

In [ ]:
# BigQuery Cell Dependencies
! python2 -m pip install --user pandas-gbq

In [ ]:
# NOTE: The RuntimeWarnings (if any) are harmless. See ContinuumIO/anaconda-issues#6678.
from pandas.io import gbq

### Configure Variables

This involves setting up the Ksonnet application as well as utility environment variables for various CLI steps.

In [ ]:
# Configuration Variables. Modify as desired.

PROJECT = 'kubeflow-dev'

# Dataflow Related Variables.
TARGET_DATASET = 'code_search'
WORKING_DIR = 'gs://kubeflow-examples/t2t-code-search/notebook-demo'
WORKER_MACHINE_TYPE = 'n1-highcpu-32'
NUM_WORKERS = 16

# DO NOT MODIFY. These are environment variables to be used in a bash shell.
%env PROJECT $PROJECT
%env TARGET_DATASET $TARGET_DATASET
%env WORKING_DIR $WORKING_DIR
%env WORKER_MACHINE_TYPE $WORKER_MACHINE_TYPE
%env NUM_WORKERS $NUM_WORKERS

###  Setup Authorization

In a Kubeflow cluster on GKE, we already have the Google Application Credentials mounted onto each Pod. We can simply point `gcloud` to activate that service account.

In [ ]:
%%bash

# Activate Service Account provided by Kubeflow.
gcloud auth activate-service-account --key-file=${GOOGLE_APPLICATION_CREDENTIALS}

Additionally, to interact with the underlying cluster, we configure `kubectl`.

In [ ]:
%%bash

kubectl config set-cluster kubeflow --server=https://kubernetes.default --certificate-authority=/var/run/secrets/kubernetes.io/serviceaccount/ca.crt
kubectl config set-credentials jupyter --token "$(cat /var/run/secrets/kubernetes.io/serviceaccount/token)"
kubectl config set-context kubeflow --cluster kubeflow --user jupyter
kubectl config use-context kubeflow

Collectively, these allow us to interact with Google Cloud Services as well as the Kubernetes Cluster directly to submit `TFJob`s and execute `Dataflow` pipelines.

### Setup Ksonnet Application

We now point the Ksonnet application to the underlying Kubernetes cluster.

In [ ]:
%%bash

cd kubeflow

# Update Ksonnet to point to the Kubernetes Cluster
ks env add code-search --context $(kubectl config current-context)

# Update the Working Directory of the application
sed -i'' "s,gs://example/prefix,${WORKING_DIR}," components/params.libsonnet

# FIXME(sanyamkapoor): This command completely replaces previous configurations.
# Hence, using string replacement in file.
# ks param set t2t-code-search workingDir ${WORKING_DIR}

## View Github Files

This is the query that is run as the first step of the Pre-Processing pipeline and is sent through a set of transformations. This is illustrative of the rows being processed in the pipeline we trigger next.

**WARNING**: The table is large and the query can take a few minutes to complete.

In [ ]:
query = """
  SELECT
    MAX(CONCAT(f.repo_name, ' ', f.path)) AS repo_path,
    c.content
  FROM
    `bigquery-public-data.github_repos.files` AS f
  JOIN
    `bigquery-public-data.github_repos.contents` AS c
  ON
    f.id = c.id
  JOIN (
      --this part of the query makes sure repo is watched at least twice since 2017
    SELECT
      repo
    FROM (
      SELECT
        repo.name AS repo
      FROM
        `githubarchive.year.2017`
      WHERE
        type="WatchEvent"
      UNION ALL
      SELECT
        repo.name AS repo
      FROM
        `githubarchive.month.2018*`
      WHERE
        type="WatchEvent" )
    GROUP BY
      1
    HAVING
      COUNT(*) >= 2 ) AS r
  ON
    f.repo_name = r.repo
  WHERE
    f.path LIKE '%.py' AND --with python extension
    c.size < 15000 AND --get rid of ridiculously long files
    REGEXP_CONTAINS(c.content, r'def ') --contains function definition
  GROUP BY
    c.content
  LIMIT
    10
"""

gbq.read_gbq(query, dialect='standard', project_id=PROJECT)

## Pre-Processing Github Files

In this step, we will run a [Google Cloud Dataflow](https://cloud.google.com/dataflow/) pipeline (based on Apache Beam). A `Python 2` module `code_search.dataflow.cli.preprocess_github_dataset` has been provided which builds an Apache Beam pipeline. A list of all possible arguments can be seen via the following command.

In [ ]:
%%bash

cd src

python2 -m code_search.dataflow.cli.preprocess_github_dataset -h

### Run the Dataflow Job for Pre-Processing

See help above for a short description of each argument. The values are being taken from environment variables defined earlier.

In [ ]:
%%bash

cd src

JOB_NAME="preprocess-github-dataset-$(date +'%Y%m%d-%H%M%S')"

python2 -m code_search.dataflow.cli.preprocess_github_dataset \
        --runner DataflowRunner \
        --project "${PROJECT}" \
        --target_dataset "${TARGET_DATASET}" \
        --data_dir "${WORKING_DIR}/data" \
        --job_name "${JOB_NAME}" \
        --temp_location "${WORKING_DIR}/dataflow/temp" \
        --staging_location "${WORKING_DIR}/dataflow/staging" \
        --worker_machine_type "${WORKER_MACHINE_TYPE}" \
        --num_workers "${NUM_WORKERS}"

When completed successfully, this should create a dataset in `BigQuery` named `target_dataset`. Additionally, it also dumps CSV files into `data_dir` which contain training samples (pairs of function and docstrings) for our Tensorflow Model. A representative set of results can be viewed using the following query.

In [ ]:
query = """
  SELECT * 
  FROM 
    {}.token_pairs
  LIMIT
    10
""".format(TARGET_DATASET)

gbq.read_gbq(query, dialect='standard', project_id=PROJECT)

This pipeline also writes a set of CSV files which contain function and docstring pairs delimited by a comma. Here, we list a subset of them.

In [ ]:
%%bash

LIMIT=10

gsutil ls ${WORKING_DIR}/data/*.csv | head -n ${LIMIT}

## Prepare Dataset for Training

We will use `t2t-datagen` to convert the transformed data above into the `TFRecord` format.

**TIP**: Use `ks show` to view the Resource Spec submitted.

In [ ]:
%%bash

cd kubeflow

ks show code-search -c t2t-code-search-datagen

In [ ]:
%%bash

cd kubeflow

ks apply code-search -c t2t-code-search-datagen

Once this job finishes, the data directory should have a vocabulary file and a list of `TFRecords` prefixed by the problem name which in our case is `github_function_docstring_extended`. Here, we list a subset of them.

In [ ]:
%%bash

LIMIT=10

gsutil ls ${WORKING_DIR}/data/vocab*
gsutil ls ${WORKING_DIR}/data/*train* | head -n ${LIMIT}

## Execute Tensorflow Training

Once, the `TFRecords` are generated, we will use `t2t-trainer` to execute the training.

In [ ]:
%%bash

cd kubeflow

ks show code-search -c t2t-code-search-trainer

In [ ]:
%%bash

cd kubeflow

ks apply code-search -c t2t-code-search-trainer

This will generate TensorFlow model checkpoints which is illustrated below.

In [ ]:
%%bash

gsutil ls ${WORKING_DIR}/output/*ckpt*

## Export Tensorflow Model

We now use `t2t-exporter` to export the `TFModel`.

In [ ]:
%%bash

cd kubeflow

ks show code-search -c t2t-code-search-exporter

In [ ]:
%%bash

cd kubeflow

ks apply code-search -c t2t-code-search-exporter

Once completed, this will generate a TensorFlow `SavedModel` which we will further use for both online (via `TF Serving`) and offline inference (via `Kubeflow Batch Prediction`).

In [ ]:
%%bash

gsutil ls ${WORKING_DIR}/output/export/Servo

## Compute Function Embeddings

In this step, we will use the exported model above to compute function embeddings via another `Dataflow` pipeline. A `Python 2` module `code_search.dataflow.cli.create_function_embeddings` has been provided for this purpose. A list of all possible arguments can be seen below.

In [ ]:
%%bash

cd src

python2 -m code_search.dataflow.cli.create_function_embeddings -h

### Configuration

First, select a Exported Model version from the `${WORKING_DIR}/output/export/Servo` as seen above. This should be name of a folder with UNIX Seconds Timestamp like `1533685294`. Below, we automatically do that by selecting the folder which represents the latest timestamp.

In [ ]:
%%bash --out EXPORT_DIR_LS

gsutil ls ${WORKING_DIR}/output/export/Servo | grep -oE "([0-9]+)/$"

In [ ]:
# WARNING: This routine will fail if no export has been completed successfully.
MODEL_VERSION = max([int(ts[:-1]) for ts in EXPORT_DIR_LS.split('\n') if ts])

# DO NOT MODIFY. These are environment variables to be used in a bash shell.
%env MODEL_VERSION $MODEL_VERSION

### Run the Dataflow Job for Function Embeddings

In [ ]:
%%bash

cd src

JOB_NAME="compute-function-embeddings-$(date +'%Y%m%d-%H%M%S')"
PROBLEM=github_function_docstring_extended

python2 -m code_search.dataflow.cli.create_function_embeddings \
        --runner DataflowRunner \
        --project "${PROJECT}" \
        --target_dataset "${TARGET_DATASET}" \
        --problem "${PROBLEM}" \
        --data_dir "${WORKING_DIR}/data" \
        --saved_model_dir "${WORKING_DIR}/output/export/Servo/${MODEL_VERSION}" \
        --job_name "${JOB_NAME}" \
        --temp_location "${WORKING_DIR}/dataflow/temp" \
        --staging_location "${WORKING_DIR}/dataflow/staging" \
        --worker_machine_type "${WORKER_MACHINE_TYPE}" \
        --num_workers "${NUM_WORKERS}"

When completed successfully, this should create another table in the same `BigQuery` dataset which contains the function embeddings for each existing data sample available from the previous Dataflow Job. Additionally, it also dumps a CSV file containing metadata for each of the function and its embeddings. A representative query result is shown below.

In [ ]:
query = """
  SELECT * 
  FROM 
    {}.function_embeddings
  LIMIT
    10
""".format(TARGET_DATASET)

gbq.read_gbq(query, dialect='standard', project_id=PROJECT)

The pipeline also generates a set of CSV files which will be useful to generate the search index.

In [ ]:
%%bash

LIMIT=10

gsutil ls ${WORKING_DIR}/data/*index*.csv | head -n ${LIMIT}

## Create Search Index

We now create the Search Index from the computed embeddings. This facilitates k-Nearest Neighbor search to for semantically similar results.

In [ ]:
%%bash

cd kubeflow

ks show code-search -c search-index-creator

In [ ]:
%%bash

cd kubeflow

ks apply code-search -c search-index-creator

Using the CSV files generated from the previous step, this creates an index using [NMSLib](https://github.com/nmslib/nmslib). A unified CSV file containing all the code examples for a human-readable reverse lookup during the query, is also created.

In [ ]:
%%bash

gsutil ls ${WORKING_DIR}/code_search_index*

## Deploy an Inference Server

We've seen offline inference during the computation of embeddings. For online inference, we deploy the exported Tensorflow model above using [Tensorflow Serving](https://www.tensorflow.org/serving/).

In [ ]:
%%bash

cd kubeflow

ks show code-search -c t2t-code-search-serving

In [ ]:
%%bash

cd kubeflow

ks apply code-search -c t2t-code-search-serving

## Deploy Search UI

We finally deploy the Search UI which allows the user to input arbitrary strings and see a list of results corresponding to semantically similar Python functions. This internally uses the inference server we just deployed.

In [ ]:
%%bash

cd kubeflow

ks show code-search -c search-index-server

In [ ]:
%%bash

cd kubeflow

ks apply code-search -c search-index-server

The service should now be available at FQDN of the Kubeflow cluster at path `/code-search/`.